The script starts by importing essential Python libraries:

In [ ]:
import numpy as np                 
import pandas as pd                   
import matplotlib.pyplot as plt       
import dm4bem                         

Flags are defined to control the behavior of the simulation:controller = True enables a simulated temperature controller.neglect_air_glass_capacity = True assumes air and glass have no thermal capacity             imposed_time_step decides if we force a fixed time step (Δt = 1800 s = 30 min).

In [ ]:
controller = True                            
neglect_air_glass_capacity = True            
imposed_time_step = False                    
Δt = 1800                                    

  The thermal circuit (a representation of the building’s thermal behavior) is loaded from a CSV file using dm4bem.file2TC.
  If the controller is activated, two conductances q6 and q14 are set to a very large value (1000 W/K) to simulate perfect indoor temperature control.

In [ ]:
TC = dm4bem.file2TC('./MODEL/TC.csv', name='', auto_number=False)
if controller:
    TC['G']['q6'] = 1e3        
    TC['G']['q14'] = 1e3 

The thermal circuit is converted into a state-space model using dm4bem.tc2ss(). This allows us to simulate the thermal dynamics of the system with matrices.

In [ ]:
As, Bs, Cs, Ds, us] = dm4bem.tc2ss(TC)

 Compute eigenvalues, max stable Δt, and choose time step



In [ ]:

λ = np.linalg.eig(As)[0]
Δtmax = 2 * min(-1 / λ)
dm4bem.print_rounded_time('Δtmax', Δtmax)

imposed_time_step = True
dt = Δt if imposed_time_step else dm4bem.round_time(Δtmax)

if dt < 10:
    raise ValueError("Time step is too small. Stopping the script.")

dm4bem.print_rounded_time('dt', dt)

Calculate settling time and total duration for simulation

In [ ]:
t_settle = 4 * max(-1 / λ)
dm4bem.print_rounded_time('t_settle', t_settle)

duration = np.ceil(t_settle / 3600) * 3600
dm4bem.print_rounded_time('duration', duration)

 Set start and end dates for simulation (May 1 to May 7, 2025)

In [ ]:
start_date = '05-01 12:00:00'
end_date = '05-07 12:00:00'
start_date = '2025-' + start_date
end_date = '2025-' + end_date
print(f'{start_date} \tstart date')
print(f'{end_date} \tend date')

 Load EPW file, get air temp and radiation, filter for period

In [ ]:
filename = './weather_data/FRA_AR_Grenoble_Airport.epw'
[data, meta] = dm4bem.read_epw(filename, coerce_year=None)

weather = data[["temp_air", "dir_n_rad", "dif_h_rad"]]
del data
weather.index = weather.index.map(lambda t: t.replace(year=2025))
weather = weather.loc[start_date:end_date]


 Get outside air temperature as input

In [ ]:
To = weather['temp_air']

 Define α, τ for walls and glass; load wall data

In [ ]:
α_wSW = 0.25
α_gSW = 0.38
τ_gSW = 0.30

wall_out = pd.read_csv('./BLDG/walls_out.csv')
w1 = wall_out[wall_out['ID'] == 'w0']
w2 = wall_out[wall_out['ID'] == 'w1']
wall_in = pd.read_csv('./BLDG/walls_in.csv')
w3 = wall_out[wall_out['ID'] == 'w0']

Compute tilted surface irradiance Etot

In [ ]:
surface_orientation = {'slope': w1['β'].values[0],
                       'azimuth': w1['γ'].values[0],
                       'latitude': 45}

rad_surf = dm4bem.sol_rad_tilt_surf(weather, surface_orientation, w1['albedo'].values[0])
Etot = rad_surf.sum(axis=1)

Calculate solar energy absorbed: Φo1, Φo2, Φi1, Φi2
Shortwave radiation properties of walls and glass are defined:

In [ ]:
Φo1 = w1['α1'].values[0] * w1['Area'].values[0] * Etot
Φo2 = w2['α1'].values[0] * w2['Area'].values[0] * Etot

S_glass = 12  # m²
Φi1 = τ_gSW * w2['α0'].values[0] * S_glass * Etot
Φi2 = τ_gSW * w3['α0'].values[0] * S_glass * Etot

Set indoor temps: 20°C during day, 16°C at night

In [ ]:
Ti_day, Ti_night = 20, 16

Ti1 = pd.Series([Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour], index=To.index)
Ti2 = pd.Series([Ti_day if 6 <= hour <= 22 else Ti_night for hour in To.index.hour], index=To.index)

 Combine all data in one DataFrame and save to CSV

In [ ]:
input_data_set = pd.DataFrame({'To': To, 'Ti1': Ti1, 'Ti2': Ti2,
                               'Φo1': Φo1, 'Φi1': Φi1, 'Φi2': Φi2,
                               'Φo2': Φo2, 'Etot': Etot})

input_data_set.to_csv('./MODEL/input_data_set.csv')